In [8]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np
pd.set_option('display.max_rows', None)
import zipfile
import os
import shutil

In [9]:
scenario_names = ['0','1','2','3','4']

In [10]:
for scenario_name in scenario_names:
    path_to_zip_file = '/home/ai6644/Malmo/Tools/Simulation results/Lolland/TRB/drt/dtm 1.5/'+\
                       scenario_name+'/log.zip'
    day_sec = int(86400)
    day_h = int(24)
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        try:
            shutil.rmtree('data/zip')
        except (FileNotFoundError, OSError) as e:
            pass
        os.mkdir('data/zip')
        zip_ref.extractall('data/zip')
        path = 'data/zip/' + os.listdir('data/zip')[0] + '/vehicle_logs'
        
    # all_files = glob.glob(path + "/vehicle_occupancy_*")
    all_files = glob.glob("data/zip/*/vehicle_logs/vehicle_occupancy_*")
    all_files.sort()

    occupancies = []
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        df['time'] = df.time.astype(int)
        df = df.drop_duplicates(keep='last', subset='time')
        occupancies.append(df)

    merged_time = pd.concat(occupancies)['time'].sort_values().drop_duplicates()
    n_cars = len(all_files)
    
    all_files = glob.glob("data/zip/*/vehicle_logs/vehicle_status_*")
    all_files.sort()

    statuses = []
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        df.columns = ['time', 'status']
        df['time'] = df.time.astype(int)
        df = df.drop_duplicates(keep='last', subset='time')
        statuses.append(df)

    for occ,stat in zip(occupancies, statuses):
        occ.set_index('time', inplace=True)
        stat.set_index('time', inplace=True)
        # idle
        occ['#passengers'].loc[(stat.status == 6).values] = -2
        # waiting outside depot
        occ['#passengers'].loc[(stat.status == 4).values] = -1

    merged_df = pd.concat(occupancies, axis=1).fillna(method='ffill')
    
    li=[]
    for i in range(0, len(merged_df)):
        li.append(merged_df.iloc[i].value_counts())
    #     li[-1].index = 8 - li[-1].index.values    
    
    day_values = pd.concat(li, axis=1).fillna(0).T.reindex(index=[_ for _ in range(0, day_sec)]).fillna(method='ffill')
    day_values = day_values[day_values.columns.to_list()[::-1]]
    day_values.columns = day_values.columns.to_list()[:-2] + ['waiting', 'idling']
    day_values2 = day_values.copy()
    day_values2['idling'] = day_values2.idling + day_values2.waiting
    day_values2 = day_values2.drop(['waiting'], axis=1)
    
    print(str(n_cars - day_values2.idling.min()).replace('.',','))

28,0
26,0
27,0
29,0
29,0


In [12]:
merged_df.max()

#passengers   -2.0
#passengers    5.0
#passengers   -2.0
#passengers    5.0
#passengers   -2.0
#passengers   -2.0
#passengers   -2.0
#passengers    5.0
#passengers    6.0
#passengers    4.0
#passengers    5.0
#passengers    5.0
#passengers    4.0
#passengers    7.0
#passengers    7.0
#passengers   -2.0
#passengers    5.0
#passengers   -2.0
#passengers    6.0
#passengers   -2.0
#passengers    6.0
#passengers    7.0
#passengers    6.0
#passengers    4.0
#passengers    6.0
#passengers    5.0
#passengers   -2.0
#passengers    5.0
#passengers   -2.0
#passengers    4.0
#passengers   -2.0
#passengers   -2.0
#passengers   -2.0
#passengers    3.0
#passengers    5.0
#passengers   -2.0
#passengers    4.0
#passengers   -2.0
#passengers   -2.0
#passengers    1.0
#passengers    5.0
#passengers    4.0
#passengers   -2.0
#passengers   -2.0
#passengers   -2.0
#passengers    4.0
#passengers    2.0
#passengers   -2.0
#passengers   -2.0
#passengers   -2.0
#passengers   -2.0
#passengers   -2.0
#passengers 